### Restart and Run All Cells

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2022
quarter = 3
today = date.today()
today_str = today.strftime("%Y-%m-%d")
today_str

'2023-02-14'

In [2]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',    
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%'
              }

In [3]:
sql = """
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2022 AND quarter <= 3) 
OR (year = 2022-1 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC


In [4]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.sample(5).style.format(format_dict)

,name,year,quarter,q_amt,Counter
41,CBG,8087,10,"2,490,900",4
135,OSP,8087,10,"2,448,105",4
12,ASIAN,8087,10,"1,020,743",4
129,NER,8087,10,"1,984,536",4
108,KYE,8087,10,"65,069",4


In [5]:
sql = """
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2022-1 AND quarter <= 3) 
OR (year = 2022-2 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC



In [6]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.sample(5).style.format(format_dict)

,name,year,quarter,q_amt,Counter
125,MEGA,8083,10,"1,877,717",4
178,SPCG,8083,10,"2,512,467",4
74,GGC,8083,10,"983,347",4
188,SUPEREIF,8083,10,"392,187",4
73,GFPT,8083,10,"617,194",4


In [7]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
15,AWC,2022,Q3,"3,414,980","-597,662","4,012,642",671.39%
198,TMW,2022,Q3,"175,994","137,823","38,171",27.70%
167,SKR,2022,Q3,"1,676,234","671,172","1,005,062",149.75%
29,BGC,2022,Q3,"508,499","550,097","-41,598",-7.56%
141,PTT,2022,Q3,"100,847,152","93,965,796","6,881,356",7.32%


In [8]:
sql = '''
DELETE FROM yr_profits 
WHERE year = %s AND quarter = "Q%s"'''
sql = sql % (year, quarter)
print(sql)

rp = conlt.execute(sql)
rp.rowcount


DELETE FROM yr_profits 
WHERE year = 2022 AND quarter = "Q3"


216

In [9]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
rcds = df_ins.values.tolist()
len(rcds)

226

In [10]:
sql = """
INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [11]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [12]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
160,SCGP,2022,Q3,"7,466,698","7,664,850","-198,152",-2.59%
66,ESSO,2022,Q3,"11,822,040","4,126,097","7,695,943",186.52%
93,JASIF,2022,Q3,"6,944,110","8,129,081","-1,184,971",-14.58%
222,WHART,2022,Q3,"1,900,269","2,308,085","-407,816",-17.67%
184,SYNEX,2022,Q3,"899,451","764,070","135,381",17.72%


In [13]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
68,FPT,2022,Q3,"1,901,129","2,714,180","-813,051",-29.96%
170,SPALI,2022,Q3,"8,880,962","6,055,750","2,825,212",46.65%
154,SAT,2022,Q3,"907,833","1,012,725","-104,892",-10.36%
159,SCCC,2022,Q3,"3,822,686","4,117,213","-294,527",-7.15%
207,TSTH,2022,Q3,"823,146","2,761,232","-1,938,086",-70.19%


In [14]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
18,BANPU,2022,Q3,"44,283,431","5,897,490","38,385,941",650.89%
187,TFFIF,2022,Q3,"1,390,654","1,086,172","304,482",28.03%
5,AIMIRT,2022,Q3,"693,015","437,639","255,376",58.35%
40,CENTEL,2022,Q3,"52,641","-3,252,627","3,305,268",101.62%
28,BEM,2022,Q3,"2,228,913","1,183,772","1,045,141",88.29%


In [15]:
final_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[final_criteria, cols].sort_values(by=["percent"], ascending=[False]).sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
60,EA,2022,Q3,"7,313,344","5,702,847","1,610,497",28.24%
3,AH,2022,Q3,"1,541,418","1,041,977","499,441",47.93%
81,HMPRO,2022,Q3,"6,339,392","5,210,457","1,128,935",21.67%
209,TTB,2022,Q3,"13,147,199","8,909,595","4,237,604",47.56%
188,TFG,2022,Q3,"3,733,094","847,771","2,885,323",340.34%


In [16]:
df_ins.loc[final_criteria, cols].sort_values(by=["name"], ascending=[True]).sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
71,GFPT,2022,Q3,"1,646,990","617,194","1,029,796",166.85%
166,SKN,2022,Q3,"814,184","498,309","315,875",63.39%
31,BH,2022,Q3,"4,004,426","776,568","3,227,858",415.66%
44,COM7,2022,Q3,"3,035,153","2,279,167","755,986",33.17%
107,LH,2022,Q3,"8,334,140","7,278,457","1,055,683",14.50%


In [17]:
df_ins.loc[final_criteria, cols].sort_values(by=["name"], ascending=[True]).sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
8,AMATA,2022,Q3,"2,296,513","1,058,096","1,238,417",117.04%
41,CHG,2022,Q3,"4,305,776","2,644,996","1,660,780",62.79%
126,OISHI,2022,Q3,"1,028,962","730,662","298,300",40.83%
201,TPIPL,2022,Q3,"7,761,502","3,930,600","3,830,902",97.46%
76,GULF,2022,Q3,"9,055,342","6,471,046","2,584,296",39.94%
